#### Using python2.7

In [ ]:
from __future__ import print_function

In [ ]:
import torch as tc

In [ ]:
a = tc.rand(5, 3)
a

In [ ]:
b = tc.ones(5, 3)
b

In [ ]:
print(a + b)
a.add_(b)

In [ ]:
print(a.size())

### Numpy Interfacing

In [ ]:
a_np = a.numpy()
a_np

In [ ]:
#memory shared for both type of vars (numpy torch)
a[0,0] = 20
print(a)
print(a_np)

In [ ]:
import numpy as np

In [ ]:
c_np = np.ones(9)
c_np = c_np.reshape(3,3)
c_tc = tc.from_numpy(c_np)
print(c_tc)
c_np

In [ ]:
c_np[0,0] = 20
print (c_np)
print (c_tc)

## AutoGrad

In [ ]:
from torch.autograd import Variable
x = tc.ones(2, 2)
x = Variable(x, requires_grad=True)
y = x + 2
y

In [ ]:
#Grad func implicitly taken
print(y.grad_fn)

In [ ]:
z = y * y * 3
o = z.mean()
o

In [ ]:
o.backward()


In [ ]:
#gradient computed for (do/dx)
print(x.grad)

In [ ]:
x.data

In [ ]:
x.data.norm()

In [ ]:
x = Variable(tc.randn(3), requires_grad = True)
x

In [ ]:
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

y

In [ ]:
print(y.grad_fn)
gradients = tc.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)


## Neural Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)    # input: 1 channel, out : 6 depth, kernel: 5 x 5
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over 2 x 2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        print(x.size()) 
        size = x.size()[1:]   #all dim except the batch dim
        print(size)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)
print(net.parameters)

In [ ]:
input = Variable(tc.randn(1, 1, 32, 32))
out = net(input)
print(out)

In [ ]:
#Zero the gradient buffers of all params and backprop with random gradients
net.zero_grad()
out.backward(tc.randn(1, 10))

In [ ]:
#Computing the loss

output = net(input)

target = Variable(tc.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

In [ ]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [ ]:
#Backprop
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

In [ ]:
#Update the weights
# weight = weight - learning_rate * gradient

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

print(loss)